# **CUSUM 1.0 BOT**
Generative AI ChatBot for Customer Support at XYZ-GoShip
# **Name : Shrawon Dey**
**email ID : d.shrawon@iitg.ac.in"**


In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import nltk
import json
import random


In [28]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
with open('Cusum1.0data.json', 'r') as f:
    data = json.load(f)

In [30]:
lemmatizer = WordNetLemmatizer()
ignore_letters = ['?', '!', '.', ',',';','+',':']

words = []
classes = []
documents = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [31]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))
print(words)
print(classes)
print(documents)

["'m", 'a', 'accept', 'account', 'address', 'am', 'an', 'any', 'arrive', 'availability', 'available', 'bad', 'bot', 'buy', 'can', 'cancel', 'change', 'check', 'code', 'complain', 'complaint', 'confirm', 'confirmation', 'contact', 'create', 'current', 'customer', 'delivery', 'detail', 'did', 'discount', 'do', 'doe', 'efficiency', 'exchange', 'failed', 'fb100', 'feedback', 'file', 'for', 'forgot', 'from', 'go', 'going', 'greeting', 'grievance', 'have', 'having', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'impressed', 'improvement', 'in', 'instruction', 'is', 'issue', 'item', 'lack', 'leave', 'link', 'location', 'long', 'me', 'method', 'modify', 'much', 'my', 'na', 'need', 'new', 'newsletter', 'not', 'of', 'option', 'order', 'password', 'pay', 'payment', 'policy', 'problem', 'process', 'product', 'promo', 'promotion', 'purchase', 'received', 'recommend', 'refund', 'reset', 'return', 'service', 'shipment', 'shipping', 'should', 'sign', 'site', 'status', 'stock', 'subscribe', 'subscription',

In [32]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    pattern_words = document[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)

# Convert to NumPy arrays *after* the loop
train_x = np.array([data[0] for data in training])
train_y = np.array([data[1] for data in training])

In [33]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
18/18 [==============================] - 1s 2ms/step - loss: 3.2848 - accuracy: 0.0225 
Epoch 2/200
18/18 [==============================] - 0s 2ms/step - loss: 3.2459 - accuracy: 0.0674
Epoch 3/200
18/18 [==============================] - 0s 2ms/step - loss: 3.2121 - accuracy: 0.0337
Epoch 4/200
18/18 [==============================] - 0s 2ms/step - loss: 3.1814 - accuracy: 0.0787
Epoch 5/200
18/18 [==============================] - 0s 2ms/step - loss: 3.1482 - accuracy: 0.1348
Epoch 6/200
18/18 [==============================] - 0s 2ms/step - loss: 3.0930 - accuracy: 0.1236
Epoch 7/200
18/18 [==============================] - 0s 2ms/step - loss: 3.0621 - accuracy: 0.2135
Epoch 8/200
18/18 [==============================] - 0s 2ms/step - loss: 3.0493 - accuracy: 0.1685
Epoch 9/200
18/18 [==============================] - 0s 2ms/step - loss: 2.9136 - accuracy: 0.3034
Epoch 10/200
18/18 [==============================] - 0s 2ms/step - loss: 2.9208 - accuracy: 0.2247
Epoch 11

In [34]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = get_response(ints, data)
    return res

In [35]:
#Here I have just portrayed an example
sample_query = "How can I reset my password?"
response = chatbot_response(sample_query)
print("Query:", sample_query)
print("Generated Response:", response)

1/1 [==============================] - 0s 86ms/step
Query: How can I reset my password?
Generated Response: To reset your password, click on 'Forgot Password' at the login screen and follow the instructions.


In [36]:
# Generate responses for all queries in the dataset
for intent in data['intents']:
    for pattern in intent['patterns']:
        response = chatbot_response(pattern)
        print(f"Query: {pattern}")
        print(f"Generated Response: {response}")
        print()

1/1 [==============================] - 0s 21ms/step
Query: Hello
Generated Response: Hello there! I am Cusum1.0, your assistant for queries related to XYZ-GoShip. How can I assist you?

1/1 [==============================] - 0s 18ms/step
Query: Hey there
Generated Response: Greetings, I am Cusum1.0, the Customer Support Bot for XYZ-GoShip. How may I help you?

1/1 [==============================] - 0s 18ms/step
Query: Greetings
Generated Response: Greetings, I am Cusum1.0, the Customer Support Bot for XYZ-GoShip. How may I help you?

1/1 [==============================] - 0s 24ms/step
Query: I need help
Generated Response: Hello there! I am Cusum1.0, your assistant for queries related to XYZ-GoShip. How can I assist you?

1/1 [==============================] - 0s 21ms/step
Query: Hi
Generated Response: Hello there! I am Cusum1.0, your assistant for queries related to XYZ-GoShip. How can I assist you?

1/1 [==============================] - 0s 20ms/step
Query: hey
Generated Response: Gr

In [45]:
import ipywidgets as widgets
from IPython.display import display

def chatbot_response(text):
    ints = predict_class(text, model)
    res = get_response(ints, data)
    return res

# Define the interactive components
input_text = widgets.Text(description="Query:", placeholder="Type your query here...")
output_text = widgets.Textarea(value='', placeholder='Chatbot response will appear here...', description='', disabled=True, layout=widgets.Layout(width='80%', height='200px'))

def on_button_click(b):
    user_input = input_text.value
    response = chatbot_response(user_input)
    new_text = f"Query: {user_input}\nResponse: {response}\n\n"
    output_text.value += new_text  # Append new response to the existing text
    input_text.value = ''  # Clear the input field

button = widgets.Button(description="Get Response")
button.on_click(on_button_click)

display(input_text)
display(button)
display(output_text)


Text(value='', description='Query:', placeholder='Type your query here...')

Button(description='Get Response', style=ButtonStyle())

Textarea(value='', disabled=True, layout=Layout(height='200px', width='80%'), placeholder='Chatbot response wi…

1/1 [==============================] - 0s 19ms/step
